# imports:

In [1]:
from os import listdir,mkdir
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist
from ShaniBA.GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from ShaniBA.TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

done1
stop
stop
done1
done
done


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'16092018'

# general plan:

plan:
    1. extract the following tables: 
        1. VDJDB
        2. cmv clones from Emersson paper
        3. McPas (disease-related, Friedman lab)
        4. more?
    2. merge all tables 
    

# extract CDR3 table from VDJdb and process:

### i tried to extract directly from the webpage but didn't succeed:

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
# url = "https://vdjdb.cdr3.net/search"
url='https://vdjdb.cdr3.net/'
df = pd.read_html(url)

### open txt file I downloaded manually:
the file was exported from https://vdjdb.cdr3.net/search on 22/2/18

In [ ]:
#OLD FILE:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/VDJDB_SearchTable-2018-02-22 12_27_28.612.txt'
VDJDBtable=pd.read_table(file1)
print VDJDBtable.shape

In [4]:
#New FILE 06/08/18
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/SearchTable-2018-09-16 14_42_48.397.tsv'
VDJDBtable=pd.read_table(file1)
print VDJDBtable.shape

(26093, 17)


database shape on sep 16th is identical to the one downloaded on August so I stay with the old one...
VDJDBtable shape is (26093, 17)

### organize table:

In [ ]:
VDJDBtable.columns.values

In [ ]:
VDJDBtable[VDJDBtable['Epitope species']=='HomoSapiens']

print len(VDJDBtable[VDJDBtable['Epitope species']=='HomoSapiens'])
print len(VDJDBtable)

In [ ]:
columns_to_keep=['CDR3','Species','Epitope species','Reference','Score']

In [ ]:
VDJDBtable_clean=VDJDBtable[columns_to_keep]

In [ ]:
print len(VDJDBtable_clean)

In [ ]:
# there are cases in which one cdr3 is linked to more than 1 species and/or Epitope species
df=VDJDBtable.groupby('CDR3')[['Species','Epitope species']].nunique()
print len(df[df['Species']>1])
df.sort_values(by='Species',ascending=False).head(10)

In [ ]:
df=VDJDBtable.groupby('CDR3')[['Species','Epitope species']].nunique()
print len(df[df['Epitope species']>1])
df.sort_values(by='Epitope species',ascending=False).head(10)

In [ ]:
VDJDBtable_organized=VDJDBtable_clean.groupby('CDR3').agg(lambda x: x.unique().tolist())

In [ ]:
print len(VDJDBtable_organized)

### save tables:

In [ ]:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/VDJDBtable_organized_%s' %cdate
VDJDBtable_organized.to_pickle(file1)

file2= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/VDJDBtable_organized_%s.xlsx' %cdate
VDJDBtable_organized.to_excel(file2)

# load CMV-related sequences from the Emersson paper:

### open file:

In [ ]:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/CMV-associated TCRbetaCDR3 based on PMID28369038.xlsx'  
CMV_TCRseq=pd.read_excel(file1)
CMV_TCRseq.head()

### organize table:

In [ ]:
CMV_TCRseq.columns.values

In [ ]:
columns_to_keep=['CDR3','Species','Epitope species','Reference','Score']

In [ ]:
CMV_TCRseq_clean=CMV_TCRseq[columns_to_keep]

In [ ]:
print len(CMV_TCRseq_clean)

In [ ]:
CMV_TCRseq_clean.Species.unique()

In [ ]:
CMV_TCRseq_clean['Epitope species'].unique()

### save tables:

In [ ]:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/CMV_TCRseq_clean_%s' %cdate
CMV_TCRseq_clean.to_pickle(file1)

file2= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/CMV_TCRseq_clean_%s.xlsx' %cdate
CMV_TCRseq_clean.to_excel(file2)

# download and process table from McPAS database:

### open the csv file I downloaded from:
http://friedmanlab.weizmann.ac.il/McPAS-TCR/
on: 22/2/18

In [ ]:
#olf file: extracted on Feb18:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/McPAS-TCR2.csv'  
McPAS_TCR2=pd.read_csv(file1)
print McPAS_TCR2.shape

McPAS_TCR2 shape is (12362, 28)

In [6]:
#new file: extracted on 230418:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/McPAS-TCR_160918.csv'  
McPAS_TCR_160918=pd.read_csv(file1)
print McPAS_TCR_160918.shape

(12868, 28)


McPAS_TCR_230418 shape is (12868, 28).
on 160918 the shape is still the same so I stay with the old one

McPAS table was updated since february 2018 - will keep on with the new table!

### organize table:

In [ ]:
McPAS_TCR_230418.columns.values

In [ ]:
columns_to_keep=['CDR3.beta.aa', 'Species', 'Category', 'Pathology','PubMed.ID']

In [ ]:
McPAS_TCR_230418_clean=McPAS_TCR_230418[columns_to_keep]
McPAS_TCR_230418_clean.head()

In [ ]:
# there are cases in which one cdr3 is linked to more than 1 species and/or 'Pathology'
df=McPAS_TCR_230418_clean.groupby('CDR3.beta.aa')[['Species','Pathology']].nunique()
print len(df[df['Species']>1])
df.sort_values(by='Species',ascending=False).head(10)

In [ ]:
df=McPAS_TCR_230418_clean.groupby('CDR3.beta.aa')[['Species','Pathology']].nunique()
print len(df[df['Pathology']>1])
df.sort_values(by='Pathology',ascending=False).head(10)

In [ ]:
McPAS_TCR_230418_organized=McPAS_TCR_230418_clean.groupby('CDR3.beta.aa').agg(lambda x: x.unique().tolist())
McPAS_TCR_230418_organized.head(10)

In [ ]:
print len(McPAS_TCR_230418_organized)

### save files:

In [ ]:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/McPAS_TCR_230418_organized_%s' %cdate
McPAS_TCR_230418_organized.to_pickle(file1)

file2= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/McPAS_TCR_230418_organized_%s.xlsx' %cdate
McPAS_TCR_230418_organized.to_excel(file2)

# merge all databases:

## load and edit CDR3 information from vdjdb:

In [ ]:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/VDJDBtable_organized_06082018'
VDJDBtable_organized=pd.read_pickle(file1)
VDJDBtable_organized.head()

In [ ]:
VDJDBtable_organized_renamed=VDJDBtable_organized
for column in VDJDBtable_organized_renamed.columns.values:
    newColumn=column+'_VDJDB'
    VDJDBtable_organized_renamed=VDJDBtable_organized_renamed.rename(columns={column:newColumn})
VDJDBtable_organized_renamed.head()

## load and edit CDR3 information from CMV_TCRseq:

In [ ]:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/CMV_TCRseq_clean_22022018'
CMV_TCRseq_clean=pd.read_pickle(file1)
CMV_TCRseq_clean.head()

In [ ]:
CMV_TCRseq_renamed=CMV_TCRseq_clean
CMV_TCRseq_renamed=CMV_TCRseq_renamed.set_index('CDR3')

for column in CMV_TCRseq_renamed.columns.values:
    newColumn=column+'_CMVpaper'
    CMV_TCRseq_renamed=CMV_TCRseq_renamed.rename(columns={column:newColumn})
CMV_TCRseq_renamed.head()

## load and edit CDR3 information from McPAS:

In [ ]:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/McPAS_TCR_230418_organized_23042018'
McPAS_TCR_230418_organized=pd.read_pickle(file1)
McPAS_TCR_230418_organized.head()

In [ ]:
McPAS_TCR_230418_renamed=McPAS_TCR_230418_organized
for column in McPAS_TCR_230418_renamed.columns.values:
    newColumn=column+'_McPAS'
    McPAS_TCR_230418_renamed=McPAS_TCR_230418_renamed.rename(columns={column:newColumn})
McPAS_TCR_230418_renamed.head()

## MERGING:

In [ ]:
CDR3identityTable=pd.merge(VDJDBtable_organized_renamed,CMV_TCRseq_renamed,how='outer',
                          left_index=True,right_index=True)
print len(VDJDBtable_organized_renamed)
print len(CMV_TCRseq_renamed)
print len(CDR3identityTable)
CDR3identityTable.head()

#last time checked:

#len(VDJDBtable_organized_renamed)=17873
#len(CMV_TCRseq_renamed)=168
#len(CDR3identityTable)=18026


In [ ]:
CDR3identityTable=pd.merge(CDR3identityTable,McPAS_TCR_230418_renamed,how='outer',
                          left_index=True,right_index=True)
print len(McPAS_TCR_230418_renamed)
print len(CDR3identityTable)
CDR3identityTable.head()

#last time checked:
#len(McPAS_TCR_230418_renamed)=9864
#len(CDR3identityTable)=23938

In [ ]:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/CDR3identityTable_%s' %cdate
CDR3identityTable.to_pickle(file1)

file2= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/CDR3identityTable_%s.xlsx' %cdate
CDR3identityTable.to_excel(file2)

In [ ]:
CDR3identityTable.head()

In [ ]:
print len(CDR3identityTable[(CDR3identityTable['Epitope species_VDJDB'].notnull())&\
                            (CDR3identityTable['Epitope species_CMVpaper'].notnull())])

#060818: common rows for VDJDB and CMV paper=19

In [ ]:
print len(CDR3identityTable[(CDR3identityTable['Epitope species_VDJDB'].notnull())&\
                            (CDR3identityTable['Species_McPAS'].notnull())])

#060818: common rows for VDJDB and McPas paper=3757

In [ ]:
print len(CDR3identityTable[(CDR3identityTable['Epitope species_VDJDB'].notnull())&\
                            (CDR3identityTable['Epitope species_CMVpaper'].notnull())&\
                           (CDR3identityTable['Species_McPAS'].notnull())])

#060818: common rows for all dbs=13

In [ ]:
import pandas as pd
import requests


# url = "http://friedmanlab.weizmann.ac.il/McPAS-TCR/session/d09a000f637ef46db5bfd0d6e74f9a55/download/downloadDB?w="
url="http://friedmanlab.weizmann.ac.il/McPAS-TCR"
response = requests.get(url)

df = pd.read_html(response.content)[0]
print(df)

In [ ]:
import pandas as pd
from lxml import html

url = "http://friedmanlab.weizmann.ac.il/McPAS-TCR/"
xpath = "//*[@id=\"mw-content-text\"]/table[3]"

tree = html.parse(url)
print tree
table = tree.xpath(xpath)
table
# [0]
# raw_html = html.tostring(table)

# dta = pd.read_html(raw_html, header=0)[0]
# dta["completed"] = 0
# del dta["Map"]

# table.make_links_absolute()
# dta["map_link"] = [i[1][0].get('href') for i in table[1:]]

# generate popped annpotation lists for McPAS and VDJDB seperately:

In [37]:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/CDR3identityTable_06082018_processed.xlsx' 
CDR3df=pd.read_excel(file1)
CDR3df.head()

,Species_VDJDB,Epitopespecies_VDJDB,Reference_VDJDB,Score_VDJDB,Species_CMVpaper,Epitopespecies_CMVpaper,Reference_CMVpaper,Score_CMVpaper,Species_McPAS,Category_McPAS,Pathology_McPAS,PubMed.ID_McPAS
CAAAAGNTGKLIF,MusMusculus,MCMV,PMID:28636592,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CAAAASGGSYIPTF,HomoSapiens,EBV,PMID:16148129,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CAAADDKIIF,HomoSapiens,YellowFeverVirus,PMID:28103239,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CAAADEEIGNQPQHF,HomoSapiens,HCV,PMID:21160049,1,NaN,NaN,NaN,NaN,Human,Pathogens,HepatitisCvirus,21160049.0
CAAAGGNNKLTF,MusMusculus,MCMV,PMID:28636592,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
print len(CDR3df)
print len(CDR3df[CDR3df['Epitopespecies_VDJDB'].notnull()])

25767
19511


In [127]:
def gen_popped_annotation_table(origDF,column):
    print 'original df length=%s' %len(origDF)
    popped=pd.DataFrame(origDF[column])
    popped=popped[popped[column].notnull()]
    try:
        popped=popped.reset_index().drop_duplicates().set_index('index')
    except:
        popped=popped.reset_index().drop_duplicates(subset='index').set_index('index')
    print 'original df length after duplcate removal =%s' %len(popped)
    for n,seq in enumerate(popped.index):
#         print popped.loc[seq,column]
#         if not pd.isnull(popped.loc[seq,column]):
        annot_list=popped.loc[seq,column].split(',')
        if n%500==0:
            print n,seq,annot_list, len(annot_list)
        if len(annot_list)>1:
            popped=popped.drop(seq,axis=0)
            for annot in annot_list:
                popped=pd.concat([popped,pd.DataFrame(index=[seq], data={column:annot})])
                    
    print 'Done!'
    print 'length of popped file =%s' %len(popped)
    file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/%s_popped.xlsx' %column
    popped.to_excel(file1)
    
    print 'some statistics:'
    print 'length of popped file =%s' %len(popped)
    print 'number of unique annotations is %s' %popped[column].nunique()
    print 'number of unique sequences is %s' %popped.index.nunique()

    print 'top annotated sequences are:'
    print popped.index.value_counts().sort_values(ascending=False).head()
    print '-------'
    print 'number of sequences per annotation:'
    print popped[column].value_counts().sort_values(ascending=False)
    
    return popped
    
    

In [57]:
origDF=CDR3df
column='Epitopespecies_VDJDB'

popped=gen_popped_annotation_table(origDF,column)

original df length=25767
original df length after duplcate removal =19509
0 CAAAAGNTGKLIF [u'MCMV'] 1
500 CAERLQTGANNLFF [u'CMV'] 1
1000 CAISQRTGGETQYF [u'CMV'] 1
1500 CAMREFTSGTYKYIF [u'HCV'] 1
2000 CASGEPGQGQDTQYF [u'InfluenzaA'] 1
2500 CASMGLGNTIYF [u'CMV'] 1
3000 CASRKAGTRIYNEQFF [u'CMV'] 1
3500 CASSAGTGFLFF [u'CMV'] 1
4000 CASSDSQNEQFF [u'EBV'] 1
4500 CASSEKGDTQYF [u'HIV-1'] 1
5000 CASSFGRVGGYTF [u'HomoSapiens'] 1
5500 CASSGDGYEQYF [u'CMV'] 1
6000 CASSHQGNSNQPQYF [u'SIV'] 1
6500 CASSIRSGIEAFF [u'InfluenzaA'] 1
7000 CASSLAPGVTNEKLFF [u'CMV'] 1
7500 CASSLFARSPLHF [u'HomoSapiens'] 1
8000 CASSLGRGQFF [u'HCV'] 1
8500 CASSLMQGQGRNGYTF [u'HCV'] 1
9000 CASSLSPSASYEQYF [u'EBV'] 1
9500 CASSLWDYGYTF [u'HomoSapiens'] 1
10000 CASSPDSNSNQPQYF [u'SIV'] 1
10500 CASSPLGANTGQLYF [u'InfluenzaA'] 1
11000 CASSPSSTGFYNEQFF [u'HTLV-1'] 1
11500 CASSQDPGGNTGQLYF [u'InfluenzaA'] 1
12000 CASSQGLNTGELFF [u'InfluenzaA'] 1
12500 CASSQVPDRGREQYF [u'HomoSapiens'] 1
13000 CASSSDINYGYTF [u'EBV'] 1
13500 CASSSRGANE

In [62]:
origDF=CDR3df
column='Pathology_McPAS'

popped=gen_popped_annotation_table(origDF,column)

original df length=25767
original df length after duplcate removal =9864
0 CAAADEEIGNQPQHF [u'HepatitisCvirus'] 1
500 CASGGSSAETLYF [u'Influenza'] 1
1000 CASRLDYNNEQF [u'RheumatoidArthritis(RA)'] 1
1500 CASSDGTANSDYTF [u'DiabetesType1'] 1
2000 CASSFKRDQPQHF [u'Multiplesclerosis(MS)'] 1
2500 CASSHRQGGTGELF [u'HepatitisCvirus'] 1
3000 CASSLASGEDTQYF [u'RheumatoidArthritis(RA)'] 1
3500 CASSLGQAYEQYF [u'EpsteinBarrvirus(EBV)', u'RheumatoidArthritis(RA)'] 2
4000 CASSLRPLDMNTEAF [u'Cytomegalovirus(CMV)'] 1
4500 CASSLYQNTNTQYF [u'DiabetesType1'] 1
5000 CASSPLKTTYEQYF [u'Melanoma'] 1
5500 CASSQASGGPINEQFF [u'Cytomegalovirus(CMV)'] 1
6000 CASSQGQGNAETLYF [u'SystemicLupusErythematosus(SLE)'] 1
6500 CASSRQAPWGRDTEAFF [u'EpsteinBarrvirus(EBV)'] 1
7000 CASSSSGPRDEQF [u'Melanoma'] 1
7500 CASSVWADPNEAF [u'Melanoma'] 1
8000 CASTGRSEAFF [u'DiabetesType1'] 1
8500 CAWSDIWENYNSPLHF [u'Influenza'] 1
9000 CSARDRTGNTIYF [u'EpsteinBarrvirus(EBV)'] 1
9500 CSVEREGGYTF [u'EpsteinBarrvirus(EBV)'] 1
Done!
length o

In [121]:
origDF=CDR3df
column='Category_McPAS'

popped=gen_popped_annotation_table(origDF,column)

original df length=25767
original df length after duplcate removal =9864
Pathogens
0 CAAADEEIGNQPQHF [u'Pathogens'] 1
Cancer
Pathogens
Autoimmune
Pathogens
Cancer
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Autoimmune
Autoimmune
Pathogens
Pathogens
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Cancer
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Allergy
Pathogens
Autoimmune
Pathogens
Pathogens
Autoimmune
Autoimmune
Pathogens


Pathogens
Pathogens
Pathogens
Allergy
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Allergy
Pathogens
Pathogens
Autoimmune
Autoimmune
Pathogens
Pathogens
Allergy
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Allergy
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Autoimmune
Pathogens
Autoimmune
Allergy
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Pathogens
Autoimmune
Cancer
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Autoimmune
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Cancer
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Cancer
Autoimmune
Pathogens
Allergy
Pathogen

Autoimmune
Autoimmune
Pathogens
Pathogens
Pathogens
Cancer
Cancer
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Cancer
Cancer
Pathogens
Pathogens
Pathogens
Cancer
Autoimmune
Autoimmune
Pathogens
Cancer
Pathogens
Pathogens
Allergy
Autoimmune
Cancer
Cancer
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Cancer
Autoimmune
Pathogens
Pathogens
Allergy
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Autoimmune
Cancer
Pathogens
Pathogens
Pathogens
Cancer
Allergy
Autoimmune
Pathogens
Other
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Autoimmune
Pathogens
Pathogens
Pathoge

Pathogens
Autoimmune
Autoimmune
Pathogens
Autoimmune
Pathogens
Cancer
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Pathogens
Cancer
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Pathogens
Pathogens
Autoimmune
Autoimmune
Pathogens
Autoimmune
Pathogens
Pathogens
Autoimmune
Autoimmune
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune,Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Autoimmune
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Autoimmune
Allergy
Path

Pathogens
Pathogens
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Other
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens,Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Autoimmune
Autoimmune
Autoimmune
Pathogens
Pathogens
Other
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Autoimmune
Pathogens
Autoimmune
Pathogens
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens,Autoimmune
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens


...

Autoimmune
Autoimmune
Pathogens
Pathogens
Other
Cancer
Autoimmune
Allergy
Cancer
Cancer
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
Cancer
Autoimmune
Pathogens
Autoimmune
Pathogens
Pathogens
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Autoimmune
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Autoimmune,Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
P

Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Cancer,Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Autoimmune
Autoimmune
Cancer
Pathogens
Cancer
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
7000 CASSSSGPRDEQF [u'Cancer'] 1
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Cancer
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Pathogens
Pathogens
Allergy
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Allergy
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens,Cancer
Allergy
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
P

Cancer
Pathogens
Autoimmune
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Allergy
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Cancer
Allergy
Pathogens
Pathogens
Pathogens
Autoimmune
Autoimmune
Pathogens
Pathogens
Pathogens
Autoimmune
Cancer
Allergy
Pathogens
Autoimmune
Cancer
Autoimmune
Autoimmune
Pathogens
Pathogens
Autoimmune
Pathogens
Autoimmune
Autoimmune
Pathogens
Cancer
Autoimmune
Allergy
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Pathogens
Allergy
Pathogens
Autoimmune
Pathogens
Autoimmune
Allergy
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Cancer,Autoimmune
Autoimmune
Pathogens
Cancer
Pathogens
Autoimmune
Cancer
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Cancer
Pathogens
Pathogens
Pa

Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Allergy
Pathogens
Pathogens
Autoimmune
Other
Autoimmune
Pathogens
Cancer
Cancer
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Other
Cancer
Cancer
Pathogens
Cancer
Other
Pathogens
Cancer
Autoimmune
Cancer
Pathogens
Cancer
Other
Other
Other
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Allergy
Autoimmune
Cancer
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Cancer
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Autoimmune
Pathogens
Autoimmune
Autoimmune
Cancer
Autoimmune
Pathogens
Pathogens
Pathogens
Pa

Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens,Autoimmune
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Other
Pathogens
Pathogens
Autoimmune
Autoimmune
Pathogens
Pathogens
Pathogens
Autoimmune
Allergy
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Autoimmune
Pathogens
Pathogens
Pathogens
Cancer
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
Autoimmune
Pathogens
Pathogens
Pathogens
P

# merge info from McPAS and VDJDB

## define general list of annotations:

In [69]:
#load annotation names and new names to convert:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/category name equalization mcPAS-vdjdb_forPython.xlsx'
nameChange=pd.read_excel(file1)
nameChange.head()



,original,new
0,Allergy,Allergy
1,Allograftrejection,Allograftrejection
2,BreastCancer,BreastCancer
3,CalcifiedAorticStenosisdisease,CalcifiedAorticStenosisdisease
4,Clearcellrenalcarcinoma,Carcinoma


In [70]:
#generate a list of tuples for conversion:
original=nameChange.original.tolist()
new=nameChange.new.tolist()

convert=zip(original,new)
convert[:5]

[(u'Allergy', u'Allergy'),
 (u'Allograftrejection', u'Allograftrejection'),
 (u'BreastCancer', u'BreastCancer'),
 (u'CalcifiedAorticStenosisdisease', u'CalcifiedAorticStenosisdisease'),
 (u'Clearcellrenalcarcinoma', u'Carcinoma')]

## merge anotations from McPAS, VDJDB:

In [132]:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/CDR3identityTable_06082018_processed.xlsx' 
CDR3df=pd.read_excel(file1)
CDR3df.head()

,Species_VDJDB,Epitopespecies_VDJDB,Reference_VDJDB,Score_VDJDB,Species_CMVpaper,Epitopespecies_CMVpaper,Reference_CMVpaper,Score_CMVpaper,Species_McPAS,Category_McPAS,Pathology_McPAS,PubMed.ID_McPAS
CAAAAGNTGKLIF,MusMusculus,MCMV,PMID:28636592,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CAAAASGGSYIPTF,HomoSapiens,EBV,PMID:16148129,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CAAADDKIIF,HomoSapiens,YellowFeverVirus,PMID:28103239,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CAAADEEIGNQPQHF,HomoSapiens,HCV,PMID:21160049,1,NaN,NaN,NaN,NaN,Human,Pathogens,HepatitisCvirus,21160049.0
CAAAGGNNKLTF,MusMusculus,MCMV,PMID:28636592,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
def arrange_annot_list(annot_list,convert):
    new_annot_list=[]
    for annot in annot_list:
        for con in convert:
            if con[0]==annot:
                new_annot=con[1]
                new_annot_list.append(new_annot)
    new_annot_list=list(set(new_annot_list))
    new_annot_list=','.join(new_annot_list)
    return new_annot_list


merged=CDR3df[['Epitopespecies_VDJDB','Pathology_McPAS']]
print len(merged)
merged=merged[(merged['Epitopespecies_VDJDB'].notnull())|(merged['Pathology_McPAS'].notnull())]
print len(merged)
print merged.head()

merged=merged.fillna('')
merged['combined annotation']=merged.apply(lambda x: ','.join(x),axis=1)
merged['combined annotation_list']=merged['combined annotation'].str.split(',')
merged['combined annotation_list_clean']=merged['combined annotation_list'].apply(lambda x: arrange_annot_list(x,convert))
# merged['combined annotation_list_clean']=merged['combined annotation_list_clean'].apply
# merged['Epitopespecies_VDJDB_list']=merged['Epitopespecies_VDJDB'].str.split(',').tolist()
# merged['Pathology_McPAS_list']=merged['Pathology_McPAS'].str.split(',').tolist()
# merged['combined annotation']=np.where(merged['Epitopespecies_VDJDB'].isnull(),merged['Pathology_McPAS'],
#                                       np.where(merged['Pathology_McPAS'].isnull(),merged['Epitopespecies_VDJDB'],
#                     (merged[['Epitopespecies_VDJDB','Pathology_McPAS']]).apply(lambda x: ','.join(str(x)),axis=1)))
#                                       ','.join([*merged['Epitopespecies_VDJDB_list'].tolist(),*merged['Pathology_McPAS_list']])))
merged.iloc[1000:1020,:]

25767
25620
                Epitopespecies_VDJDB  Pathology_McPAS
CAAAAGNTGKLIF                   MCMV              NaN
CAAAASGGSYIPTF                   EBV              NaN
CAAADDKIIF          YellowFeverVirus              NaN
CAAADEEIGNQPQHF                  HCV  HepatitisCvirus
CAAAGGNNKLTF                    MCMV              NaN


,Epitopespecies_VDJDB,Pathology_McPAS,combined annotation,combined annotation_list,combined annotation_list_clean
CAISESGGRVDEQYF,HIV-1,Humanimmunodeficiencyvirus(HIV),"HIV-1,Humanimmunodeficiencyvirus(HIV)","[HIV-1, Humanimmunodeficiencyvirus(HIV)]",HIV
CAISESGQALITYEQYF,HCV,,"HCV,","[HCV, ]",HCV
CAISESGRANTEAFF,HomoSapiens,,"HomoSapiens,","[HomoSapiens, ]",HomoSapiens
CAISESGSPEQYF,CMV,Cytomegalovirus(CMV),"CMV,Cytomegalovirus(CMV)","[CMV, Cytomegalovirus(CMV)]",CMV
CAISESGYGGPPGANVLTF,HIV-1,,"HIV-1,","[HIV-1, ]",HIV
CAISESGYRGPPGANVLTF,HIV-1,,"HIV-1,","[HIV-1, ]",HIV
CAISESMGTGTQETQYF,"DENV1,DENV3/4",,"DENV1,DENV3/4,","[DENV1, DENV3/4, ]",DENV
CAISESNRGYEQYF,,RheumatoidArthritis(RA),",RheumatoidArthritis(RA)","[, RheumatoidArthritis(RA)]",RheumatoidArthritis
CAISESPDMLMGNEQFF,CMV,,"CMV,","[CMV, ]",CMV
CAISESPQSYEQYF,HCV,,"HCV,","[HCV, ]",HCV


## generate popped file for the merged column:

In [136]:
merged.index.nunique()

25618

In [137]:
origDF=merged
column='combined annotation_list_clean'

popped=gen_popped_annotation_table(origDF,column)

original df length=25620
original df length after duplcate removal =25618
0 CAAAAGNTGKLIF [u'CMV'] 1
500 CAENIGGGSQGNLIF [u'Influenza'] 1
1000 CAISESGGRVDEQYF [u'HIV'] 1
1500 CALTGGDYGYTF [u'HIV'] 1
2000 CASARLAGGTDEQFF [u'HIV'] 1
2500 CASGPLGGDNQPQHF [u'Influenza'] 1
3000 CASMGNSAGANVLTF [u'CMV'] 1
3500 CASRGDLEGYTF [u'Influenza'] 1
4000 CASRRGTSGTGELFF [u'HIV'] 1
4500 CASSARAFPEGNQPQHF [u'HIV'] 1
5000 CASSDRTGGKEPQYF [u'SIV'] 1
5500 CASSEGGEQYF [u'Influenza'] 1
6000 CASSFAGTDTQYF [u'HomoSapiens'] 1
6500 CASSFPGTDTQYF [u'CMV'] 1
7000 CASSGGAEVFF [u'Influenza'] 1
7500 CASSHGQLQETQYF [u'Melanoma'] 1
8000 CASSILGGADVQFF [u'Influenza'] 1
8500 CASSKIAGGPYTQYF [u'Influenza'] 1
9000 CASSLAVIRTDTQYF [u'DiabetesType1'] 1
9500 CASSLETGTEAFF [u'HIV'] 1
10000 CASSLGMFDQPQHF [u'Influenza'] 1
10500 CASSLIENTEAFF [u'M.tuberculosis'] 1
11000 CASSLPGDYAEQFF [u'CMV'] 1
11500 CASSLSGTGELFF [u'HCV'] 1
12000 CASSLVANTQYF [u'CMV'] 1
12500 CASSMGQGWEQYF [u'Influenza'] 1
13000 CASSPGARGIDEQFF [u'HIV'] 1
1350